In [1]:
!python -V

Python 3.11.0


In [2]:
import pandas as pd

import pickle


from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [3]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
        df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [5]:
df_jan = pd.read_parquet("./data/yellow_tripdata_2022-01.parquet")
df_feb = pd.read_parquet("./data/yellow_tripdata_2022-02.parquet")

Question 1: Number of columns in Jan 2022 Yellow Taxi Trip data *

In [6]:
print(len(df_jan.columns))

19


Question 2: Standard deviation of the trips duration in Jan 2022 Yellow Taxi Trip data

In [7]:
df_jan['duration'] = df_jan.tpep_dropoff_datetime - df_jan.tpep_pickup_datetime
df_jan.duration = df_jan.duration.apply(lambda td: td.total_seconds() / 60)
print(round(df_jan["duration"].std(), 2))

46.45


Question 3: Fraction of the records left after dropping the outliers

In [8]:
non_outliers = df_jan.query("duration>=1 and duration<=60")
print(round(100*len(non_outliers)/len(df_jan)))

98


Question 4: Dimensionality after OHE

In [10]:
df_train = read_dataframe('./data/yellow_tripdata_2022-01.parquet')
df_val = read_dataframe('./data/yellow_tripdata_2022-02.parquet')

categorical = ['PULocationID', 'DOLocationID']

dv = DictVectorizer()

train_dicts = df_train[categorical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical ].to_dict(orient='records')
X_val = dv.transform(val_dicts)

print(X_train.shape[1])

515


Question 5: RMSE on train

In [11]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)
print(round(mean_squared_error(y_train, y_pred, squared=False), 2))

6.99


Question 6: RMSE on validation

In [12]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)
print(round(mean_squared_error(y_val, y_pred, squared=False), 2))

7.79
